In [1]:
import math
import numpy as np
import random
from random import randint
from gurobipy import *
#import pandas as pd
from random import seed
import matplotlib.pyplot as plt
import geopy.distance

In [2]:
#path = r"C:/Users/Devika Kabe/Documents/Model_brainstorming/"
path = "/home/dkabe/Model_brainstorming/"

In [3]:
f = open(path + 'Input_Data/product_distributions.txt', 'r')
text = f.read()
f.close()
distributions = text.split('\n')

In [4]:
for i in distributions:
    exec(i.lstrip().rstrip())

In [5]:
distribution_mat = np.array([[p1_mean, p1_sd],
                             [p2_mean, p2_sd],
                             [p3_mean, p3_sd],
                             [p4_mean, p4_sd]])

In [6]:
Products = 2
Outsourced = 2

In [7]:
## 2 MPs, 3 DCs, 4 MZs
## 2 Products, 2 Outsourced Facilities 

Manufacturing_plants = 2
Distribution = 3
num_Scenarios = (2**Manufacturing_plants)*(2**Distribution) 
Market = 1

# Cost of opening
f_i = [7500, 12500]
f_j = [7500, 12500, 17500]

# Unit cost of manufacturing product
np.random.seed(0)
Manufacturing_costs = np.random.uniform(0,1, (Manufacturing_plants,Products))

# Unit cost of transporting m from plant to DC
np.random.seed(0)
Transportation_i_j = np.random.uniform(0,1, (Products, Manufacturing_plants, Distribution))

# Unit cost of transporting m from DC to Market Zone
np.random.seed(0)
Transportation_j_k = np.random.uniform(0,1, (Products, Distribution, Market))

# Plant Capacities: Bigger capacities for the more expensive ones
np.random.seed(0)
Capacities_i = np.zeros(Manufacturing_plants) # in volume (metres cubed)
Capacities_i[0] = 5000
Capacities_i[1] = 10000

Capacities_j = np.zeros(Distribution) # in volume (metres cubed)
Capacities_j[0] = 5000
Capacities_j[1] = 10000
Capacities_j[2] = 15000

Capacities_l = np.random.randint(400, 500, (Products,Outsourced)) # in terms of products 

# Cost of purchasing product m from supplier l 
np.random.seed(0)
levels = 2
Supplier_cost = np.zeros((levels, Products, Outsourced))
Supplier_cost[0] = np.random.uniform(17, 20, (Products, Outsourced))
Supplier_cost[1] = np.random.uniform(20, 23, (Products, Outsourced))

# Cost of transporting product m from outsourced facility l to j
np.random.seed(0)
T_O_DC = np.random.uniform(1, 2, (Products, Outsourced, Distribution))

# Cost of shipping product m from outsourced facility l to k
np.random.seed(0)
T_O_MZ = np.random.uniform(2, 3,(Products, Outsourced, Market))

# Product volume 
np.random.seed(0)
volume = np.random.uniform(1, 2,(Products))

# unit cost of lost sales 
np.random.seed(1)
lost_sales = np.random.uniform(20, 25, (Market,Products))

# Demand
demand = np.zeros((num_Scenarios, Products, Market))
np.random.seed(0)
for s in range(num_Scenarios):
    for k in range(Market):
        for m in range(Products):
            demand[s][m][k] = np.round(np.random.normal(distribution_mat[m][0], distribution_mat[m][1]))

# Compatibilities
compatibility = np.ones((Products, Manufacturing_plants))

In [8]:
# save files 
np.savetxt(path + 'Input_Data/Instance_1/OpenMP_1.txt', f_i)
np.savetxt(path + 'Input_Data/Instance_1/OpenDC_1.txt', f_j)
np.savetxt(path + 'Input_Data/Instance_1/Manufacturing_1.txt', Manufacturing_costs)
np.savetxt(path + 'Input_Data/Instance_1/TransMPDC_1.txt', Transportation_i_j.reshape((Products*Manufacturing_plants, Distribution)))
np.savetxt(path + 'Input_Data/Instance_1/TransDCMZ_1.txt', Transportation_j_k.reshape((Products*Distribution, Market)))
np.savetxt(path + 'Input_Data/Instance_1/CapacitiesMP_1.txt', Capacities_i)
np.savetxt(path + 'Input_Data/Instance_1/CapacitiesDC_1.txt', Capacities_j)
np.savetxt(path + 'Input_Data/Instance_1/CapacitiesOutsource_1.txt', Capacities_l)
np.savetxt(path + 'Input_Data/Instance_1/SupplierCost_1.txt', Supplier_cost.reshape((levels*Products, Outsourced)))
np.savetxt(path + 'Input_Data/Instance_1/TransSupplierDC_1.txt', T_O_DC.reshape((Products*Outsourced, Distribution)))
np.savetxt(path + 'Input_Data/Instance_1/TransSupplierMZ_1.txt', T_O_MZ.reshape((Products*Outsourced, Market)))
np.savetxt(path + 'Input_Data/Instance_1/Volume_1.txt', volume)
np.savetxt(path + 'Input_Data/Instance_1/LostSales_1.txt', lost_sales)
np.savetxt(path + 'Input_Data/Instance_1/Demand_1.txt', demand.reshape((num_Scenarios*Products, Market)))
np.savetxt(path + 'Input_Data/Instance_1/Compatibilities_1.txt', compatibility)

In [9]:
## 3 MPs, 4 DCs, 2 MZs

num_Scenarios = 128

Manufacturing_plants = 3
Distribution = 4
Market = 2
Outsourced = 2

# Cost of opening
f_i = [7500, 12500, 17500]
f_j = [7500, 12500, 17500, 7500]

# Unit cost of manufacturing product
np.random.seed(0)
Manufacturing_costs = np.random.uniform(0,1, (Manufacturing_plants,Products))

# Unit cost of transporting m from plant to DC
np.random.seed(0)
Transportation_i_j = np.random.uniform(0,1, (Products, Manufacturing_plants, Distribution))

# Unit cost of transporting m from DC to Market Zone
np.random.seed(0)
Transportation_j_k = np.random.uniform(0,1, (Products, Distribution, Market))

# Plant Capacities: Bigger capacities for the more expensive ones
np.random.seed(0)
Capacities_i = np.zeros(Manufacturing_plants) # in volume (metres cubed)
Capacities_i[0] = 5000
Capacities_i[1] = 10000
Capacities_i[2] = 15000
Capacities_j = np.zeros(Distribution) # in volume (metres cubed)
Capacities_j[0] = 5000
Capacities_j[1] = 10000
Capacities_j[2] = 15000
Capacities_j[3] = 5000
Capacities_l = np.random.randint(800, 1000, (Products,Outsourced)) # in terms of products 

# Cost of purchasing product m from supplier l 
np.random.seed(0)
levels = 2
Supplier_cost = np.zeros((levels, Products, Outsourced))
Supplier_cost[0] = np.random.uniform(17, 20, (Products, Outsourced))
Supplier_cost[1] = np.random.uniform(20, 23, (Products, Outsourced))

# Cost of transporting product m from outsourced facility l to j
np.random.seed(0)
T_O_DC = np.random.uniform(1, 2, (Products, Outsourced, Distribution))

# Cost of shipping product m from outsourced facility l to k
np.random.seed(0)
T_O_MZ = np.random.uniform(2, 3,(Products, Outsourced, Market))

# Product volume 
np.random.seed(0)
volume = np.random.uniform(1, 2,(Products))

# unit cost of lost sales 
np.random.seed(1)
lost_sales = np.random.uniform(20, 25, (Market,Products))

# Demand
demand = np.zeros((num_Scenarios, Products, Market))
np.random.seed(0)
for s in range(num_Scenarios):
    for k in range(Market):
        for m in range(Products):
            demand[s][m][k] = np.round(np.random.normal(distribution_mat[m][0], distribution_mat[m][1]))

# Compatibilities
compatibility = np.ones((Products, Manufacturing_plants))

In [10]:
np.savetxt(path + 'Input_Data/Instance_2/OpenMP_2.txt', f_i)
np.savetxt(path + 'Input_Data/Instance_2/OpenDC_2.txt', f_j)
np.savetxt(path + 'Input_Data/Instance_2/Manufacturing_2.txt', Manufacturing_costs)
np.savetxt(path + 'Input_Data/Instance_2/TransMPDC_2.txt', Transportation_i_j.reshape((Products*Manufacturing_plants, Distribution)))
np.savetxt(path + 'Input_Data/Instance_2/TransDCMZ_2.txt', Transportation_j_k.reshape((Products*Distribution, Market)))
np.savetxt(path + 'Input_Data/Instance_2/CapacitiesMP_2.txt', Capacities_i)
np.savetxt(path + 'Input_Data/Instance_2/CapacitiesDC_2.txt', Capacities_j)
np.savetxt(path + 'Input_Data/Instance_2/CapacitiesOutsource_2.txt', Capacities_l)
np.savetxt(path +'Input_Data/Instance_2/SupplierCost_2.txt', Supplier_cost.reshape((levels*Products, Outsourced)))
np.savetxt(path + 'Input_Data/Instance_2/TransSupplierDC_2.txt', T_O_DC.reshape((Products*Outsourced, Distribution)))
np.savetxt(path + 'Input_Data/Instance_2/TransSupplierMZ_2.txt', T_O_MZ.reshape((Products*Outsourced, Market)))
np.savetxt(path + 'Input_Data/Instance_2/Volume_2.txt', volume)
np.savetxt(path + 'Input_Data/Instance_2/LostSales_2.txt', lost_sales)
np.savetxt(path + 'Input_Data/Instance_2/Demand_2.txt', demand.reshape((num_Scenarios*Products, Market)))
np.savetxt(path + 'Input_Data/Instance_2/Compatibilities_2.txt', compatibility)

In [11]:
## 4 MPs, 6 DCs, 3 MZs

num_Scenarios = 200
Manufacturing_plants = 4
Distribution = 6
Market = 3
Products = 2
Outsourced = 2
# Cost of opening
f_i = [7500, 12500, 17500, 7500]
f_j = [7500, 12500, 17500, 7500, 12500, 17500]

# Unit cost of manufacturing product
np.random.seed(0)
Manufacturing_costs = np.random.uniform(0,1, (Manufacturing_plants,Products))

# Unit cost of transporting m from plant to DC
np.random.seed(0)
Transportation_i_j = np.random.uniform(0,1, (Products, Manufacturing_plants, Distribution))

# Unit cost of transporting m from DC to Market Zone
np.random.seed(0)
Transportation_j_k = np.random.uniform(0,1, (Products, Distribution, Market))

# Plant Capacities: Bigger capacities for the more expensive ones
np.random.seed(0)
Capacities_i = np.zeros(Manufacturing_plants) # in volume (metres cubed)
Capacities_i[0] = 5000
Capacities_i[1] = 10000
Capacities_i[2] = 15000
Capacities_i[3] = 5000
Capacities_j = np.zeros(Distribution) # in volume (metres cubed)
Capacities_j[0] = 5000
Capacities_j[1] = 10000
Capacities_j[2] = 15000
Capacities_j[3] = 5000
Capacities_j[4] = 10000
Capacities_j[5] = 15000
Capacities_l = np.random.randint(1200, 1500, (Products,Outsourced)) # in terms of products 

# Cost of purchasing product m from supplier l 
np.random.seed(0)
levels = 2
Supplier_cost = np.zeros((levels, Products, Outsourced))
Supplier_cost[0] = np.random.uniform(17, 20, (Products, Outsourced))
Supplier_cost[1] = np.random.uniform(20, 23, (Products, Outsourced))

# Cost of transporting product m from outsourced facility l to j
np.random.seed(0)
T_O_DC = np.random.uniform(1, 2, (Products, Outsourced, Distribution))

# Cost of shipping product m from outsourced facility l to k
np.random.seed(0)
T_O_MZ = np.random.uniform(2, 3,(Products, Outsourced, Market))

# Product volume 
np.random.seed(0)
volume = np.random.uniform(1, 2,(Products))

# unit cost of lost sales 
np.random.seed(1)
lost_sales = np.random.uniform(20, 25, (Market,Products))

# Demand
demand = np.zeros((num_Scenarios, Products, Market))
np.random.seed(0)
for s in range(num_Scenarios):
    for k in range(Market):
        for m in range(Products):
            demand[s][m][k] = np.round(np.random.normal(distribution_mat[m][0], distribution_mat[m][1]))
# Compatibilities
compatibility = np.ones((Products, Manufacturing_plants))

In [12]:
np.savetxt(path + 'Input_Data/Instance_3/OpenMP_3.txt', f_i)
np.savetxt(path + 'Input_Data/Instance_3/OpenDC_3.txt', f_j)
np.savetxt(path + 'Input_Data/Instance_3/Manufacturing_3.txt', Manufacturing_costs)
np.savetxt(path + 'Input_Data/Instance_3/TransMPDC_3.txt', Transportation_i_j.reshape((Products*Manufacturing_plants, Distribution)))
np.savetxt(path + 'Input_Data/Instance_3/TransDCMZ_3.txt', Transportation_j_k.reshape((Products*Distribution, Market)))
np.savetxt(path + 'Input_Data/Instance_3/CapacitiesMP_3.txt', Capacities_i)
np.savetxt(path + 'Input_Data/Instance_3/CapacitiesDC_3.txt', Capacities_j)
np.savetxt(path + 'Input_Data/Instance_3/CapacitiesOutsource_3.txt', Capacities_l)
np.savetxt(path + 'Input_Data/Instance_3/SupplierCost_3.txt', Supplier_cost.reshape((levels*Products, Outsourced)))
np.savetxt(path + 'Input_Data/Instance_3/TransSupplierDC_3.txt', T_O_DC.reshape((Products*Outsourced, Distribution)))
np.savetxt(path + 'Input_Data/Instance_3/TransSupplierMZ_3.txt', T_O_MZ.reshape((Products*Outsourced, Market)))
np.savetxt(path + 'Input_Data/Instance_3/Volume_3.txt', volume)
np.savetxt(path + 'Input_Data/Instance_3/LostSales_3.txt', lost_sales)
np.savetxt(path + 'Input_Data/Instance_3/Demand_3.txt', demand.reshape((num_Scenarios*Products, Market)))
np.savetxt(path + 'Input_Data/Instance_3/Compatibilities_3.txt', compatibility)

In [13]:
## 6 MPs, 8 DCs, 5 MZs

num_Scenarios = 200
Manufacturing_plants = 6
Distribution = 8
Market = 5

# Cost of opening
f_i = [7500, 12500, 17500, 7500, 12500, 17500]
f_j = [7500, 12500, 17500, 7500, 12500, 17500, 7500, 12500]
# Unit cost of manufacturing product
np.random.seed(0)
Manufacturing_costs = np.random.uniform(0,1, (Manufacturing_plants,Products))

# Unit cost of transporting m from plant to DC
np.random.seed(0)
Transportation_i_j = np.random.uniform(0,1, (Products, Manufacturing_plants, Distribution))

# Unit cost of transporting m from DC to Market Zone
np.random.seed(0)
Transportation_j_k = np.random.uniform(0,1, (Products, Distribution, Market))

# Plant Capacities: Bigger capacities for the more expensive ones
np.random.seed(0)
Capacities_i = np.zeros(Manufacturing_plants) # in volume (metres cubed)
Capacities_i[0] = 5000
Capacities_i[1] = 10000
Capacities_i[2] = 15000
Capacities_i[3] = 5000
Capacities_i[4] = 10000
Capacities_i[5] = 15000
Capacities_j = np.zeros(Distribution) # in volume (metres cubed)
Capacities_j[0] = 5000
Capacities_j[1] = 10000
Capacities_j[2] = 15000
Capacities_j[3] = 5000
Capacities_j[4] = 10000
Capacities_j[5] = 15000
Capacities_j[6] = 5000
Capacities_j[7] = 10000
Capacities_l = np.random.randint(2000, 2500, (Products,Outsourced)) # in terms of products 

# Cost of purchasing product m from supplier l 
np.random.seed(0)
levels = 2
Supplier_cost = np.zeros((levels, Products, Outsourced))
Supplier_cost[0] = np.random.uniform(17, 20, (Products, Outsourced))
Supplier_cost[1] = np.random.uniform(20, 23, (Products, Outsourced))

# Cost of transporting product m from outsourced facility l to j
np.random.seed(0)
T_O_DC = np.random.uniform(1, 2, (Products, Outsourced, Distribution))

# Cost of shipping product m from outsourced facility l to k
np.random.seed(0)
T_O_MZ = np.random.uniform(2, 3,(Products, Outsourced, Market))

# Product volume 
np.random.seed(0)
volume = np.random.uniform(1, 2,(Products))

# unit cost of lost sales 
np.random.seed(1)
lost_sales = np.random.uniform(20, 25, (Market,Products))

# Demand
demand = np.zeros((num_Scenarios, Products, Market))
np.random.seed(0)
for s in range(num_Scenarios):
    for k in range(Market):
        for m in range(Products):
            demand[s][m][k] = np.round(np.random.normal(distribution_mat[m][0], distribution_mat[m][1]))
            
# Compatibilities
compatibility = np.ones((Products, Manufacturing_plants))

In [14]:
np.savetxt(path + 'Input_Data/Instance_4/OpenMP_4.txt', f_i)
np.savetxt(path + 'Input_Data/Instance_4/OpenDC_4.txt', f_j)
np.savetxt(path + 'Input_Data/Instance_4/Manufacturing_4.txt', Manufacturing_costs)
np.savetxt(path + 'Input_Data/Instance_4/TransMPDC_4.txt', Transportation_i_j.reshape((Products*Manufacturing_plants, Distribution)))
np.savetxt(path + 'Input_Data/Instance_4/TransDCMZ_4.txt', Transportation_j_k.reshape((Products*Distribution, Market)))
np.savetxt(path + 'Input_Data/Instance_4/CapacitiesMP_4.txt', Capacities_i)
np.savetxt(path + 'Input_Data/Instance_4/CapacitiesDC_4.txt', Capacities_j)
np.savetxt(path + 'Input_Data/Instance_4/CapacitiesOutsource_4.txt', Capacities_l)
np.savetxt(path + 'Input_Data/Instance_4/SupplierCost_4.txt', Supplier_cost.reshape((levels*Products, Outsourced)))
np.savetxt(path + 'Input_Data/Instance_4/TransSupplierDC_4.txt', T_O_DC.reshape((Products*Outsourced, Distribution)))
np.savetxt(path + 'Input_Data/Instance_4/TransSupplierMZ_4.txt', T_O_MZ.reshape((Products*Outsourced, Market)))
np.savetxt(path + 'Input_Data/Instance_4/Volume_4.txt', volume)
np.savetxt(path + 'Input_Data/Instance_4/LostSales_4.txt', lost_sales)
np.savetxt(path + 'Input_Data/Instance_4/Demand_4.txt', demand.reshape((num_Scenarios*Products, Market)))
np.savetxt(path + 'Input_Data/Instance_4/Compatibilities_4.txt', compatibility)

## Mimic real-life scenarios

In [15]:
# Distribution centres and market zones don't change based on instances 

with open (path + "Input_Data/DC_locations.txt") as f:
    DC_locations = [tuple(map(float, i.split('\t'))) for i in f]

with open (path + "Input_Data/market_locations.txt") as f:
    market_locations = [tuple(map(float, i.split('\t'))) for i in f]

# Distribution Centres to Market Zones 
distances_j_k = []
for loc1 in range(len(DC_locations)):
    temp_dist = []
    for loc2 in range(len(market_locations)):
        temp_dist.append(float(str(geopy.distance.geodesic(DC_locations[loc1], market_locations[loc2]))[:-3]))
    distances_j_k.append(temp_dist)

## Cleaning Products

In [16]:
# open location data

In [17]:
with open (path + "Input_Data/Instance_5/MP_locations_5.txt") as f:
    MP_locations = [tuple(map(float, i.split('\t'))) for i in f]

In [18]:
with open(path + "Input_Data/Instance_5/outsource_locations_5.txt") as f:
    outsource_locations = [tuple(map(float, i.split('\t'))) for i in f]

In [19]:
# compute geodesic distances 

In [20]:
# Plants to Distribution Centres
distances_i_j = []
for loc1 in range(len(MP_locations)):
    temp_dist = []
    for loc2 in range(len(DC_locations)):
        temp_dist.append(float(str(geopy.distance.geodesic(MP_locations[loc1], DC_locations[loc2]))[:-3]))
    distances_i_j.append(temp_dist)

In [21]:
# Outsourced Facilities to Distribution Centres
distances_l_j = []
for loc1 in range(len(outsource_locations)):
    temp_dist = []
    for loc2 in range(len(DC_locations)):
        temp_dist.append(float(str(geopy.distance.geodesic(outsource_locations[loc1], DC_locations[loc2]))[:-3]))
    distances_l_j.append(temp_dist)

In [22]:
# Outsourced Facilities to Market Zones
distances_l_k = []
for loc1 in range(len(outsource_locations)):
    temp_dist = []
    for loc2 in range(len(market_locations)):
        temp_dist.append(float(str(geopy.distance.geodesic(outsource_locations[loc1], market_locations[loc2]))[:-3]))
    distances_l_k.append(temp_dist)

In [23]:
# means and sds of products
# p1_mean = 205500
# p2_mean = 202500
# p3_mean = 199500

# p1_sd = 3300
# p2_sd = 8250
# p3_sd = 1650

p1_mean = 103500
p2_mean = 101500
p3_mean = 99750

p1_sd = 1650
p2_sd = 4125
p3_sd = 2000

In [24]:
distribution_mat = [[p1_mean, p1_sd],
                  [p2_mean, p2_sd],
                  [p3_mean, p3_sd]]

In [25]:
# 6 MPs, 4 DCs, 29 MZs
Products = 3
Outsourced = 3
num_Scenarios = 128
Manufacturing_plants = 6
Distribution = 4
Market = 29

# Cost of opening/establishing/operating 
f_i = [2070000, 2000000, 1656000]*2
f_j = [2070000, 2000000, 1656000, 1800000]

# In-house produce + transport 

np.random.seed(0)
Manufacturing_costs = np.random.uniform(0.1,0.2, (Manufacturing_plants,Products))

Transportation_i_j = np.zeros((Products, Manufacturing_plants, Distribution))
for m in range(Products):
    for i in range(Manufacturing_plants):
        for j in range(Distribution):
            Transportation_i_j[m][i][j] = distances_i_j[i][j]/10000

Transportation_j_k = np.zeros((Products, Distribution, Market))
for m in range(Products):
    for j in range(Distribution):
        for k in range(Market):
            Transportation_j_k[m][j][k] = distances_j_k[j][k]/1000

# Capacities of plants in squared feet

Capacities_i = [103500, 100000, 82800]*2

Capacities_j = [103500, 100000, 82800, 90000]

# Quantity supplier can provide 

Capacities_l = np.zeros((Products, Outsourced))
Capacities_l[0][0] = 2750000
Capacities_l[1][1] = 2800000
Capacities_l[2][2] = 2900000

# Cost of purchasing from suppliers

levels = 2
Supplier_cost = np.zeros((levels, Products, Outsourced))
Supplier_cost[0][0][0] = 10
Supplier_cost[0][1][1] = 5
Supplier_cost[0][2][2] = 7

Supplier_cost[1][0][0] = 11
Supplier_cost[1][1][1] = 6
Supplier_cost[1][2][2] = 8

# Cost of transporting from suppliers

T_O_DC = np.zeros((Products, Outsourced, Distribution))
for m in range(Products):
    for l in range(Outsourced):
        for j in range(Distribution):
            T_O_DC[m][l][j] = distances_l_j[l][j]/1000

T_O_MZ = np.zeros((Products, Outsourced, Market))
for m in range(Products):
    for l in range(Outsourced):
        for k in range(Market):
            T_O_MZ[m][l][k] = distances_l_k[l][k]/1000

# volume of products

volume = np.random.uniform(0.01, 0.03,(Products))

# unit cost of lost sales 

lost_sales = [[10, 5, 7]]*Market


# demand of each product/ scenario
demand = np.zeros((num_Scenarios, Products, Market))
np.random.seed(0)
for s in range(num_Scenarios):
    for k in range(Market):
        for m in range(Products):
            demand[s][m][k] = np.round(np.random.normal(distribution_mat[m][0], distribution_mat[m][1]))

# Compatibilities
compatibility = np.ones((Products, Manufacturing_plants))



In [26]:
# save everything to instance folder  

np.savetxt(path + 'Input_Data/Instance_5/OpenMP_5.txt', f_i)
np.savetxt(path + 'Input_Data/Instance_5/OpenDC_5.txt', f_j)
np.savetxt(path + 'Input_Data/Instance_5/Manufacturing_5.txt', Manufacturing_costs)
np.savetxt(path + 'Input_Data/Instance_5/TransMPDC_5.txt', Transportation_i_j.reshape((Products*Manufacturing_plants, Distribution)))
np.savetxt(path + 'Input_Data/Instance_5/TransDCMZ_5.txt', Transportation_j_k.reshape((Products*Distribution, Market)))
np.savetxt(path + 'Input_Data/Instance_5/CapacitiesMP_5.txt', Capacities_i)
np.savetxt(path + 'Input_Data/Instance_5/CapacitiesDC_5.txt', Capacities_j)
np.savetxt(path + 'Input_Data/Instance_5/CapacitiesOutsource_5.txt', Capacities_l)
np.savetxt(path + 'Input_Data/Instance_5/SupplierCost_5.txt', Supplier_cost.reshape((levels*Products, Outsourced)))
np.savetxt(path + 'Input_Data/Instance_5/TransSupplierDC_5.txt', T_O_DC.reshape((Products*Outsourced, Distribution)))
np.savetxt(path + 'Input_Data/Instance_5/TransSupplierMZ_5.txt', T_O_MZ.reshape((Products*Outsourced, Market)))
np.savetxt(path + 'Input_Data/Instance_5/Volume_5.txt', volume)
np.savetxt(path + 'Input_Data/Instance_5/LostSales_5.txt', lost_sales)
np.savetxt(path + 'Input_Data/Instance_5/Demand_5.txt', demand.reshape((num_Scenarios*Products, Market)))
np.savetxt(path + 'Input_Data/Instance_5/Compatibilities_5.txt', compatibility)

## Meat Products

In [27]:
# open location data
with open (path + "Input_Data/Instance_6/MP_locations_6.txt") as f:
    MP_locations = [tuple(map(float, i.split('\t'))) for i in f]

with open(path + "Input_Data/Instance_6/outsource_locations_6.txt") as f:
    outsource_locations = [tuple(map(float, i.split('\t'))) for i in f]

In [28]:
# Plants to Distribution Centres
distances_i_j = []
for loc1 in range(len(MP_locations)):
    temp_dist = []
    for loc2 in range(len(DC_locations)):
        temp_dist.append(float(str(geopy.distance.geodesic(MP_locations[loc1], DC_locations[loc2]))[:-3]))
    distances_i_j.append(temp_dist)

In [29]:
# Outsourced Facilities to Distribution Centres
distances_l_j = []
for loc1 in range(len(outsource_locations)):
    temp_dist = []
    for loc2 in range(len(DC_locations)):
        temp_dist.append(float(str(geopy.distance.geodesic(outsource_locations[loc1], DC_locations[loc2]))[:-3]))
    distances_l_j.append(temp_dist)

In [30]:
# Outsourced Facilities to Market Zones
distances_l_k = []
for loc1 in range(len(outsource_locations)):
    temp_dist = []
    for loc2 in range(len(market_locations)):
        temp_dist.append(float(str(geopy.distance.geodesic(outsource_locations[loc1], market_locations[loc2]))[:-3]))
    distances_l_k.append(temp_dist)

In [31]:
p1_mean = 137000
p2_mean = 135000
p3_mean = 133000

p1_sd = 2650
p2_sd = 8125
p3_sd = 4000

distribution_mat = [[p1_mean, p1_sd],
                  [p2_mean, p2_sd],
                  [p3_mean, p3_sd]]

In [32]:
## 6 MPs, 4 DCs, 29 MZs
Products = 3
Outsourced = 3
num_Scenarios = 200
Manufacturing_plants = 6
Distribution = 4
Market = 29

# Cost of opening/establishing/operating 
f_i = [2070000, 2000000, 1656000]*2
f_j = [2070000, 2000000, 1656000, 1800000]

# In-house produce + transport 

np.random.seed(0)
Manufacturing_costs = np.random.uniform(0.1,0.2, (Manufacturing_plants,Products))

Transportation_i_j = np.zeros((Products, Manufacturing_plants, Distribution))
for m in range(Products):
    for i in range(Manufacturing_plants):
        for j in range(Distribution):
            #if i == 0:
            #    Transportation_i_j[m][i][j] = distances_i_j[i][j]/10000
            #else: 
            Transportation_i_j[m][i][j] = distances_i_j[i][j]/10000


Transportation_j_k = np.zeros((Products, Distribution, Market))
for m in range(Products):
    for j in range(Distribution):
        for k in range(Market):
            Transportation_j_k[m][j][k] = distances_j_k[j][k]/1000

# Capacities of plants in squared feet

Capacities_i = [103500, 100000, 82800]*2

Capacities_j = [103500, 100000, 82800, 90000]

# Quantity supplier can provide 

Capacities_l = np.zeros((Products, Outsourced))
Capacities_l[0][2] = 1250000
Capacities_l[1][1] = 1300000
Capacities_l[2][0] = 1400000

# Cost of purchasing from suppliers

levels = 2
Supplier_cost = np.zeros((levels, Products, Outsourced))
Supplier_cost[0][0][2] = 11
Supplier_cost[0][1][1] = 9
Supplier_cost[0][2][0] = 7

Supplier_cost[1][0][2] = 12
Supplier_cost[1][1][1] = 10
Supplier_cost[1][2][0] = 8

# Cost of transporting from suppliers

T_O_DC = np.zeros((Products, Outsourced, Distribution))
for m in range(Products):
    for l in range(Outsourced):
        for j in range(Distribution):
            T_O_DC[m][l][j] = distances_l_j[l][j]/1000

T_O_MZ = np.zeros((Products, Outsourced, Market))
for m in range(Products):
    for l in range(Outsourced):
        for k in range(Market):
            T_O_MZ[m][l][k] = distances_l_k[l][k]/1000

# volume of products

volume = np.random.uniform(0.01, 0.02,(Products))

# unit cost of lost sales 

lost_sales = [[11, 9, 7]]*Market


# demand of each product/ scenario
demand = np.zeros((num_Scenarios, Products, Market))
np.random.seed(0)
for s in range(num_Scenarios):
    for k in range(Market):
        for m in range(Products):
            demand[s][m][k] = np.round(np.random.normal(distribution_mat[m][0], distribution_mat[m][1]))

# Compatibilities
compatibility = [[0, 1, 0, 1, 1, 0],
                 [1, 1, 0, 1, 0, 0],
                 [0, 0, 1, 0, 0, 1]]

In [33]:
np.savetxt(path + 'Input_Data/Instance_6/OpenMP_6.txt', f_i)
np.savetxt(path + 'Input_Data/Instance_6/OpenDC_6.txt', f_j)
np.savetxt(path + 'Input_Data/Instance_6/Manufacturing_6.txt', Manufacturing_costs)
np.savetxt(path + 'Input_Data/Instance_6/TransMPDC_6.txt', Transportation_i_j.reshape((Products*Manufacturing_plants, Distribution)))
np.savetxt(path + 'Input_Data/Instance_6/TransDCMZ_6.txt', Transportation_j_k.reshape((Products*Distribution, Market)))
np.savetxt(path + 'Input_Data/Instance_6/CapacitiesMP_6.txt', Capacities_i)
np.savetxt(path + 'Input_Data/Instance_6/CapacitiesDC_6.txt', Capacities_j)
np.savetxt(path + 'Input_Data/Instance_6/CapacitiesOutsource_6.txt', Capacities_l)
np.savetxt(path + 'Input_Data/Instance_6/SupplierCost_6.txt', Supplier_cost.reshape((levels*Products, Outsourced)))
np.savetxt(path + 'Input_Data/Instance_6/TransSupplierDC_6.txt', T_O_DC.reshape((Products*Outsourced, Distribution)))
np.savetxt(path + 'Input_Data/Instance_6/TransSupplierMZ_6.txt', T_O_MZ.reshape((Products*Outsourced, Market)))
np.savetxt(path + 'Input_Data/Instance_6/Volume_6.txt', volume)
np.savetxt(path + 'Input_Data/Instance_6/LostSales_6.txt', lost_sales)
np.savetxt(path + 'Input_Data/Instance_6/Demand_6.txt', demand.reshape((num_Scenarios*Products, Market)))
np.savetxt(path + 'Input_Data/Instance_6/Compatibilities_6.txt', compatibility)